![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# 🧠 Section 3: Memory Architecture - From Stateless RAG to Stateful Conversations

**⏱️ Estimated Time:** 45-60 minutes

## 🎯 Learning Objectives

By the end of this notebook, you will:

1. **Understand** why memory is essential for context engineering
2. **Implement** working memory for conversation continuity
3. **Use** long-term memory for persistent user knowledge
4. **Integrate** memory with your Section 2 RAG system
5. **Build** a complete memory-enhanced course advisor

---

## 🔗 Bridge from Sections 1 & 2

### **Section 1: The Four Context Types**

Recall the four context types from Section 1:

1. **System Context** (Static) - Role, instructions, guidelines
2. **User Context** (Dynamic, User-Specific) - Profile, preferences, goals
3. **Conversation Context** (Dynamic, Session-Specific) - **← Memory enables this!**
4. **Retrieved Context** (Dynamic, Query-Specific) - RAG results

### **Section 2: Stateless RAG**

Your Section 2 RAG system was **stateless**:

```python
def rag_query(query, student_profile):
    # 1. Search courses (Retrieved Context)
    courses = course_manager.search(query)

    # 2. Assemble context (System + User + Retrieved)
    context = assemble_context(system_prompt, student_profile, courses)

    # 3. Generate response
    response = llm.invoke(context)

    # ❌ No conversation history stored
    # ❌ Each query is independent
    # ❌ Can't reference previous messages
```

**The Problem:** Every query starts from scratch. No conversation continuity.

---

## 🚨 The Grounding Problem

**Grounding** means understanding what users are referring to. Natural conversation is full of references:

### **Without Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers supervised learning..."

User: "What are its prerequisites?"
Agent: ❌ "What does 'its' refer to? Please specify which course."

User: "The course we just discussed!"
Agent: ❌ "I don't have access to previous messages. Which course?"
```

**This is a terrible user experience.**

### **With Memory:**

```
User: "Tell me about CS401"
Agent: "CS401 is Machine Learning. It covers..."
[Stores: User asked about CS401]

User: "What are its prerequisites?"
Agent: [Checks memory: "its" = CS401]
Agent: ✅ "CS401 requires CS201 and MATH301"

User: "Can I take it?"
Agent: [Checks memory: "it" = CS401, checks student transcript]
Agent: ✅ "You've completed CS201 but still need MATH301"
```

**Now the conversation flows naturally!**

---

## 🧠 Two Types of Memory

### **1. Working Memory (Session-Scoped)**

**What:** Conversation messages from the current session

**Purpose:** Reference resolution, conversation continuity

**Lifetime:** Session duration (e.g., 1 hour TTL)

**Example:**
```
Session: session_123
Messages:
  1. User: "Tell me about CS401"
  2. Agent: "CS401 is Machine Learning..."
  3. User: "What are its prerequisites?"
  4. Agent: "CS401 requires CS201 and MATH301"
```

### **2. Long-term Memory (Cross-Session)**

**What:** Persistent facts, preferences, goals

**Purpose:** Personalization across sessions

**Lifetime:** Permanent (until explicitly deleted)

**Example:**
```
User: student_sarah
Memories:
  - "Prefers online courses over in-person"
  - "Major: Computer Science, focus on AI/ML"
  - "Goal: Graduate Spring 2026"
  - "Completed: CS101, CS201, MATH301"
```

---

## 🏗️ Memory Architecture

We'll use **Redis Agent Memory Server** - a production-ready dual-memory system:

**Working Memory:**
- Session-scoped conversation context
- Automatic extraction to long-term storage
- TTL-based expiration

**Long-term Memory:**
- Vector-indexed for semantic search
- Automatic deduplication
- Three types: semantic (facts), episodic (events), message

**Why Agent Memory Server?**
- Production-ready (handles thousands of users)
- Redis-backed (fast, scalable)
- Automatic memory management (extraction, deduplication)
- Semantic search built-in

---

## 📦 Setup

### **What We're Importing:**

- **Section 2 components** - `redis_config`, `CourseManager`, models
- **Agent Memory Server client** - `MemoryAPIClient` for memory operations
- **LangChain** - `ChatOpenAI` for LLM interaction

### **Why:**

- Build on Section 2's RAG foundation
- Add memory capabilities without rewriting everything
- Use production-ready memory infrastructure


In [ ]:
# Setup: Import components
import os
import sys
import asyncio
from typing import List, Dict, Any, Optional
from datetime import datetime
from dotenv import load_dotenv

# Load environment
load_dotenv()
sys.path.append('../../reference-agent')

# Import Section 2 components
from redis_context_course.redis_config import redis_config
from redis_context_course.course_manager import CourseManager
from redis_context_course.models import (
    Course, StudentProfile, DifficultyLevel,
    CourseFormat, Semester
)

# Import LangChain
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Import Agent Memory Server client
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    from agent_memory_client.models import WorkingMemory, MemoryMessage, ClientMemoryRecord
    MEMORY_SERVER_AVAILABLE = True
    print("✅ Agent Memory Server client available")
except ImportError:
    MEMORY_SERVER_AVAILABLE = False
    print("⚠️  Agent Memory Server not available")
    print("📝 Install with: pip install agent-memory-client")
    print("🚀 Start server: See reference-agent/README.md")

# Verify environment
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY not found. Please set in .env file.")
else:
    print("✅ OPENAI_API_KEY found")

print(f"\n🔧 Environment Setup:")
print(f"   OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"   REDIS_URL: {os.getenv('REDIS_URL', 'redis://localhost:6379')}")
print(f"   AGENT_MEMORY_URL: {os.getenv('AGENT_MEMORY_URL', 'http://localhost:8088')}")
print(f"   Memory Server: {'✓ Available' if MEMORY_SERVER_AVAILABLE else '✗ Not available'}")


### 🎯 What We Just Did

**Successfully Imported:**
- ✅ **Section 2 RAG components** - `redis_config`, `CourseManager`, models
- ✅ **Agent Memory Server client** - Production-ready memory system
- ✅ **Environment verified** - OpenAI API key, Redis, Memory Server

**Why This Matters:**
- We're **building on Section 2's foundation** (not starting from scratch)
- **Agent Memory Server** provides scalable, persistent memory
- **Same Redis University domain** for consistency

---

## 🔧 Initialize Components


In [ ]:
# Initialize components
course_manager = CourseManager()
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)

# Initialize Memory Client
if MEMORY_SERVER_AVAILABLE:
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8088"),
        default_namespace="redis_university"
    )
    memory_client = MemoryAPIClient(config=config)
    print("🧠 Memory Client Initialized")
    print(f"   Base URL: {config.base_url}")
    print(f"   Namespace: {config.default_namespace}")
else:
    memory_client = None
    print("⚠️  Running without Memory Server (limited functionality)")

# Create a sample student profile (reusing Section 2 pattern)
sarah = StudentProfile(
    name="Sarah Chen",
    email="sarah.chen@university.edu",
    major="Computer Science",
    year=2,
    interests=["machine learning", "data science", "algorithms"],
    completed_courses=["CS101", "CS201"],
    current_courses=["MATH301"],
    preferred_format=CourseFormat.ONLINE,
    preferred_difficulty=DifficultyLevel.INTERMEDIATE
)

print(f"\n👤 Student Profile: {sarah.name}")
print(f"   Major: {sarah.major}")
print(f"   Interests: {', '.join(sarah.interests)}")


### 💡 Key Insight

We're reusing:
- ✅ **Same `CourseManager`** from Section 2
- ✅ **Same `StudentProfile`** model
- ✅ **Same Redis configuration**

We're adding:
- ✨ **Memory Client** for conversation history
- ✨ **Working Memory** for session context
- ✨ **Long-term Memory** for persistent knowledge

---

## 📚 Part 1: Working Memory Fundamentals

### **What is Working Memory?**

Working memory stores **conversation messages** for the current session. It enables:

✅ **Reference resolution** - "it", "that course", "the one you mentioned"
✅ **Context continuity** - Each message builds on previous messages
✅ **Natural conversations** - Users don't repeat themselves

### **How It Works:**

```
Turn 1: Load working memory (empty) → Process query → Save messages
Turn 2: Load working memory (1 exchange) → Process query → Save messages
Turn 3: Load working memory (2 exchanges) → Process query → Save messages
```

Each turn has access to all previous messages in the session.

---

## 📚 Part 2: Long-term Memory Fundamentals

### **What is Long-term Memory?**

Long-term memory stores **persistent facts, preferences, and goals** across sessions. It enables:

✅ **Personalization** - Remember user preferences across conversations
✅ **Knowledge accumulation** - Build understanding over time
✅ **Semantic search** - Find relevant memories using natural language

### **Memory Types:**

1. **Semantic** - Facts and knowledge ("Prefers online courses")
2. **Episodic** - Events and experiences ("Enrolled in CS101 on 2024-09-01")
3. **Message** - Important conversation excerpts

### **How It Works:**

```
Session 1: User shares preferences → Store in long-term memory
Session 2: User asks for recommendations → Search long-term memory → Personalized response
Session 3: User updates preferences → Update long-term memory
```

Long-term memory persists across sessions and is searchable via semantic vector search.

---

## 🧪 Hands-On: Long-term Memory in Action

Let's store and search long-term memories.


In [ ]:
# Long-term Memory Demo
async def longterm_memory_demo():
    """Demonstrate long-term memory for persistent knowledge"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"

    print("=" * 80)
    print("🧪 LONG-TERM MEMORY DEMO: Persistent Knowledge")
    print("=" * 80)

    # Step 1: Store semantic memories (facts)
    print("\n📍 STEP 1: Storing Semantic Memories (Facts)")
    print("-" * 80)

    semantic_memories = [
        "Student prefers online courses over in-person classes",
        "Student's major is Computer Science with focus on AI/ML",
        "Student wants to graduate in Spring 2026",
        "Student prefers morning classes, no classes on Fridays",
        "Student has completed CS101 and CS201",
        "Student is currently taking MATH301"
    ]

    for memory_text in semantic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=student_id,
            memory_type="semantic",
            topics=["preferences", "academic_info"]
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ Stored: {memory_text}")

    # Step 2: Store episodic memories (events)
    print("\n📍 STEP 2: Storing Episodic Memories (Events)")
    print("-" * 80)

    episodic_memories = [
        "Student enrolled in CS101 on 2024-09-01",
        "Student completed CS101 with grade A on 2024-12-15",
        "Student asked about machine learning courses on 2024-09-20"
    ]

    for memory_text in episodic_memories:
        memory_record = ClientMemoryRecord(
            text=memory_text,
            user_id=student_id,
            memory_type="episodic",
            topics=["enrollment", "courses"]
        )
        await memory_client.create_long_term_memory([memory_record])
        print(f"   ✅ Stored: {memory_text}")

    # Step 3: Search long-term memory with semantic queries
    print("\n📍 STEP 3: Searching Long-term Memory")
    print("-" * 80)

    search_queries = [
        "What does the student prefer?",
        "What courses has the student completed?",
        "What is the student's major?"
    ]

    for query in search_queries:
        print(f"\n   🔍 Query: '{query}'")
        results = await memory_client.search_long_term_memory(
            text=query,
            user_id=student_id,
            limit=3
        )

        if results.memories:
            print(f"   📚 Found {len(results.memories)} relevant memories:")
            for i, memory in enumerate(results.memories[:3], 1):
                print(f"      {i}. {memory.text}")
        else:
            print("   ⚠️  No memories found")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Long-term memory enables persistent knowledge!")
    print("=" * 80)

# Run the demo
await longterm_memory_demo()


### 🎯 What Just Happened?

**Step 1: Stored Semantic Memories**
- Facts about preferences ("prefers online courses")
- Academic information ("major is Computer Science")
- Goals ("graduate Spring 2026")

**Step 2: Stored Episodic Memories**
- Events ("enrolled in CS101 on 2024-09-01")
- Experiences ("completed CS101 with grade A")

**Step 3: Searched with Natural Language**
- Query: "What does the student prefer?"
- Results: Memories about preferences (online courses, morning classes)
- **Semantic search** finds relevant memories even without exact keyword matches

**💡 Key Insight:** Long-term memory enables **personalization** and **knowledge accumulation** across sessions.

---

## 🔗 Part 3: Integrating Memory with RAG

Now let's combine **working memory** + **long-term memory** + **RAG** from Section 2.

### **The Complete Picture:**

```
User Query
    ↓
1. Load Working Memory (conversation history)
2. Search Long-term Memory (user preferences, facts)
3. RAG Search (relevant courses)
4. Assemble Context (System + User + Conversation + Retrieved)
5. Generate Response
6. Save Working Memory (updated conversation)
```

This gives us **all four context types** from Section 1:
- ✅ System Context (static instructions)
- ✅ User Context (profile + long-term memories)
- ✅ Conversation Context (working memory)
- ✅ Retrieved Context (RAG results)

---

## 🏗️ Building the Memory-Enhanced RAG System

Let's build a complete function that integrates everything.


In [ ]:
# Memory-Enhanced RAG Function
async def memory_enhanced_rag_query(
    user_query: str,
    student_profile: StudentProfile,
    session_id: str,
    top_k: int = 3
) -> str:
    """
    Complete memory-enhanced RAG query.

    Combines:
    - Working memory (conversation history)
    - Long-term memory (user preferences, facts)
    - RAG (semantic search for courses)

    Args:
        user_query: User's question
        student_profile: Student profile (User Context)
        session_id: Session ID for working memory
        top_k: Number of courses to retrieve

    Returns:
        Agent's response
    """

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Using simplified RAG.")
        # Fallback to Section 2 RAG
        courses = course_manager.search(user_query, limit=top_k)
        context = f"Student: {student_profile.name}\nQuery: {user_query}\nCourses: {[c.course_code for c in courses]}"
        messages = [
            SystemMessage(content="You are a helpful course advisor."),
            HumanMessage(content=context)
        ]
        return llm.invoke(messages).content

    student_id = student_profile.email.split('@')[0]

    # Step 1: Load working memory (conversation history)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    # Step 2: Search long-term memory (user preferences, facts)
    longterm_results = await memory_client.search_long_term_memory(
        text=user_query,
        user_id=student_id,
        limit=5
    )

    longterm_memories = [m.text for m in longterm_results.memories] if longterm_results.memories else []

    # Step 3: RAG search (relevant courses)
    courses = course_manager.search(user_query, limit=top_k)

    # Step 4: Assemble context (all four context types!)

    # System Context
    system_prompt = """You are a Redis University course advisor.

Your role:
- Help students find and enroll in courses
- Provide personalized recommendations
- Answer questions about courses, prerequisites, schedules

Guidelines:
- Use conversation history to resolve references ("it", "that course")
- Use long-term memories to personalize recommendations
- Be helpful, supportive, and encouraging
- If you don't know something, say so"""

    # User Context (profile + long-term memories)
    user_context = f"""Student Profile:
- Name: {student_profile.name}
- Major: {student_profile.major}
- Year: {student_profile.year}
- Interests: {', '.join(student_profile.interests)}
- Completed: {', '.join(student_profile.completed_courses)}
- Current: {', '.join(student_profile.current_courses)}
- Preferred Format: {student_profile.preferred_format.value}
- Preferred Difficulty: {student_profile.preferred_difficulty.value}"""

    if longterm_memories:
        user_context += f"\n\nLong-term Memories:\n" + "\n".join([f"- {m}" for m in longterm_memories])

    # Retrieved Context (RAG results)
    retrieved_context = "Relevant Courses:\n"
    for i, course in enumerate(courses, 1):
        retrieved_context += f"\n{i}. {course.course_code}: {course.title}"
        retrieved_context += f"\n   Description: {course.description}"
        retrieved_context += f"\n   Difficulty: {course.difficulty_level.value}"
        retrieved_context += f"\n   Format: {course.format.value}"
        retrieved_context += f"\n   Credits: {course.credits}"
        if course.prerequisites:
            prereqs = [p.course_code for p in course.prerequisites]
            retrieved_context += f"\n   Prerequisites: {', '.join(prereqs)}"
        retrieved_context += "\n"

    # Build messages with all context types
    messages = [
        SystemMessage(content=system_prompt)
    ]

    # Add conversation history (Conversation Context)
    for msg in working_memory.messages:
        if msg.role == "user":
            messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            messages.append(AIMessage(content=msg.content))

    # Add current query with assembled context
    current_message = f"""{user_context}

{retrieved_context}

User Query: {user_query}"""

    messages.append(HumanMessage(content=current_message))

    # Step 5: Generate response
    response = llm.invoke(messages).content

    # Step 6: Save working memory (updated conversation)
    working_memory.messages.extend([
        MemoryMessage(role="user", content=user_query),
        MemoryMessage(role="assistant", content=response)
    ])

    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    return response


### 🎯 What This Function Does

**Integrates All Four Context Types:**

1. **System Context** - Role, instructions, guidelines (static)
2. **User Context** - Profile + long-term memories (dynamic, user-specific)
3. **Conversation Context** - Working memory messages (dynamic, session-specific)
4. **Retrieved Context** - RAG search results (dynamic, query-specific)

**Memory Operations:**

1. **Load** working memory (conversation history)
2. **Search** long-term memory (relevant facts)
3. **Search** courses (RAG)
4. **Assemble** all context types
5. **Generate** response
6. **Save** working memory (updated conversation)

**Why This Matters:**

- ✅ **Stateful conversations** - Remembers previous messages
- ✅ **Personalized responses** - Uses long-term memories
- ✅ **Reference resolution** - Resolves "it", "that course", etc.
- ✅ **Complete context** - All four context types working together

---

## 🧪 Hands-On: Complete Memory-Enhanced RAG

Let's test the complete system with a multi-turn conversation.


In [ ]:
# Complete Memory-Enhanced RAG Demo
async def complete_demo():
    """Demonstrate complete memory-enhanced RAG system"""

    session_id = f"session_{sarah.email.split('@')[0]}_complete"

    print("=" * 80)
    print("🧪 COMPLETE DEMO: Memory-Enhanced RAG System")
    print("=" * 80)
    print(f"\n👤 Student: {sarah.name}")
    print(f"📧 Session: {session_id}")

    # Turn 1: Initial query
    print("\n" + "=" * 80)
    print("📍 TURN 1: Initial Query")
    print("=" * 80)

    query_1 = "I'm interested in machine learning courses"
    print(f"\n👤 User: {query_1}")

    response_1 = await memory_enhanced_rag_query(
        user_query=query_1,
        student_profile=sarah,
        session_id=session_id,
        top_k=3
    )

    print(f"\n🤖 Agent: {response_1}")

    # Turn 2: Follow-up with pronoun reference
    print("\n" + "=" * 80)
    print("📍 TURN 2: Follow-up with Pronoun Reference")
    print("=" * 80)

    query_2 = "What are the prerequisites for the first one?"
    print(f"\n👤 User: {query_2}")

    response_2 = await memory_enhanced_rag_query(
        user_query=query_2,
        student_profile=sarah,
        session_id=session_id,
        top_k=3
    )

    print(f"\n🤖 Agent: {response_2}")

    # Turn 3: Another follow-up
    print("\n" + "=" * 80)
    print("📍 TURN 3: Another Follow-up")
    print("=" * 80)

    query_3 = "Do I meet those prerequisites?"
    print(f"\n👤 User: {query_3}")

    response_3 = await memory_enhanced_rag_query(
        user_query=query_3,
        student_profile=sarah,
        session_id=session_id,
        top_k=3
    )

    print(f"\n🤖 Agent: {response_3}")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Memory-enhanced RAG enables natural conversations!")
    print("=" * 80)

# Run the complete demo
await complete_demo()


### 🎯 What Just Happened?

**Turn 1:** "I'm interested in machine learning courses"
- System searches courses
- Finds ML-related courses
- Responds with recommendations
- **Saves conversation to working memory**

**Turn 2:** "What are the prerequisites for **the first one**?"
- System loads working memory (Turn 1)
- Resolves "the first one" → first course mentioned in Turn 1
- Responds with prerequisites
- **Saves updated conversation**

**Turn 3:** "Do I meet **those prerequisites**?"
- System loads working memory (Turns 1-2)
- Resolves "those prerequisites" → prerequisites from Turn 2
- Checks student's completed courses (from profile)
- Responds with personalized answer
- **Saves updated conversation**

**💡 Key Insight:** Memory + RAG = **Natural, stateful, personalized conversations**

---

## 🎓 Key Takeaways

### **1. Memory Solves the Grounding Problem**

Without memory, agents can't resolve references:
- ❌ "What are **its** prerequisites?" → Agent doesn't know what "its" refers to
- ✅ With working memory → Agent resolves "its" from conversation history

### **2. Two Types of Memory Serve Different Purposes**

**Working Memory (Session-Scoped):**
- Conversation messages from current session
- Enables reference resolution and conversation continuity
- TTL-based (expires after session ends)

**Long-term Memory (Cross-Session):**
- Persistent facts, preferences, goals
- Enables personalization across sessions
- Searchable via semantic vector search

### **3. Memory Completes the Four Context Types**

From Section 1, we learned about four context types. Memory enables two of them:

1. **System Context** (Static) - ✅ Section 2
2. **User Context** (Dynamic, User-Specific) - ✅ Section 2 + Long-term Memory
3. **Conversation Context** (Dynamic, Session-Specific) - ✨ **Working Memory**
4. **Retrieved Context** (Dynamic, Query-Specific) - ✅ Section 2 RAG

### **4. Memory + RAG = Complete Context Engineering**

The integration pattern:
```
1. Load working memory (conversation history)
2. Search long-term memory (user facts)
3. RAG search (relevant documents)
4. Assemble all context types
5. Generate response
6. Save working memory (updated conversation)
```

This gives us **stateful, personalized, context-aware conversations**.

### **5. Agent Memory Server is Production-Ready**

Why use Agent Memory Server instead of simple in-memory storage:
- ✅ **Scalable** - Redis-backed, handles thousands of users
- ✅ **Automatic** - Extracts important facts to long-term storage
- ✅ **Semantic search** - Vector-indexed memory retrieval
- ✅ **Deduplication** - Prevents redundant memories
- ✅ **TTL management** - Automatic expiration of old sessions

### **6. LangChain is Sufficient for Memory + RAG**

We didn't need LangGraph for this section because:
- Simple linear flow (load → search → generate → save)
- No conditional branching or complex state management
- No tool calling required

**LangGraph becomes necessary in Section 4** when we add tools and multi-step workflows.

---

## 🚀 What's Next?

### 🛠️ Section 4: Tool Selection & Agentic Workflows

Now that you have **memory-enhanced RAG**, you'll add **tools** to create a complete agent:

**Tools You'll Add:**
- `search_courses` - Semantic search (you already have this!)
- `get_course_details` - Fetch specific course information
- `check_prerequisites` - Verify student eligibility
- `enroll_course` - Register student for a course
- `store_memory` - Explicitly save important facts
- `search_memories` - Query long-term memory

**Why LangGraph in Section 4:**
- **Tool calling** - Agent decides which tools to use
- **Conditional branching** - Different paths based on tool results
- **State management** - Track tool execution across steps
- **Error handling** - Retry failed tool calls

**The Complete Picture:**

```
Section 1: Context Engineering Fundamentals
    ↓
Section 2: RAG (Retrieved Context)
    ↓
Section 3: Memory (Conversation Context + Long-term Knowledge)
    ↓
Section 4: Tools + Agents (Complete Agentic System)
```

By Section 4, you'll have a **complete course advisor agent** that:
- ✅ Remembers conversations (working memory)
- ✅ Knows user preferences (long-term memory)
- ✅ Searches courses (RAG)
- ✅ Takes actions (tools)
- ✅ Makes decisions (agentic workflow)

---

## 💪 Practice Exercises

### **Exercise 1: Cross-Session Personalization**

Modify the `memory_enhanced_rag_query` function to:
1. Store user preferences in long-term memory when mentioned
2. Use those preferences in future sessions
3. Test with two different sessions for the same student

**Hint:** Look for phrases like "I prefer...", "I like...", "I want..." and store them as semantic memories.

### **Exercise 2: Memory-Aware Filtering**

Enhance the RAG search to use long-term memories as filters:
1. Search long-term memory for preferences (format, difficulty, schedule)
2. Apply those preferences as filters to `course_manager.search()`
3. Compare results with and without memory-aware filtering

**Hint:** Use the `filters` parameter in `course_manager.search()`.

### **Exercise 3: Conversation Summarization**

Implement a function that summarizes long conversations:
1. When working memory exceeds 10 messages, summarize the conversation
2. Store the summary in long-term memory
3. Clear old messages from working memory (keep only recent 4)
4. Test that reference resolution still works with summarized history

**Hint:** Use the LLM to generate summaries, then store as semantic memories.

### **Exercise 4: Multi-User Memory Management**

Create a simple CLI that:
1. Supports multiple students (different user IDs)
2. Maintains separate working memory per session
3. Maintains separate long-term memory per user
4. Demonstrates cross-session continuity for each user

**Hint:** Use different `session_id` and `user_id` for each student.

### **Exercise 5: Memory Search Quality**

Experiment with long-term memory search:
1. Store 20+ diverse memories for a student
2. Try different search queries
3. Analyze which memories are retrieved
4. Adjust memory text to improve search relevance

**Hint:** More specific memory text leads to better semantic search results.

---

## 📝 Summary

### **What You Learned:**

1. **The Grounding Problem** - Why agents need memory to resolve references
2. **Working Memory** - Session-scoped conversation history for continuity
3. **Long-term Memory** - Cross-session persistent knowledge for personalization
4. **Memory Integration** - Combining memory with Section 2's RAG system
5. **Complete Context Engineering** - All four context types working together
6. **Production Architecture** - Using Agent Memory Server for scalable memory

### **What You Built:**

- ✅ Working memory demo (multi-turn conversations)
- ✅ Long-term memory demo (persistent knowledge)
- ✅ Complete memory-enhanced RAG system
- ✅ Integration of all four context types

### **Key Functions:**

- `memory_enhanced_rag_query()` - Complete memory + RAG pipeline
- `working_memory_demo()` - Demonstrates conversation continuity
- `longterm_memory_demo()` - Demonstrates persistent knowledge
- `complete_demo()` - End-to-end multi-turn conversation

### **Architecture Pattern:**

```
User Query
    ↓
Load Working Memory (conversation history)
    ↓
Search Long-term Memory (user facts)
    ↓
RAG Search (relevant courses)
    ↓
Assemble Context (System + User + Conversation + Retrieved)
    ↓
Generate Response
    ↓
Save Working Memory (updated conversation)
```

### **From Section 2 to Section 3:**

**Section 2 (Stateless RAG):**
- ❌ No conversation history
- ❌ Each query independent
- ❌ Can't resolve references
- ✅ Retrieves relevant documents

**Section 3 (Memory-Enhanced RAG):**
- ✅ Conversation history (working memory)
- ✅ Multi-turn conversations
- ✅ Reference resolution
- ✅ Persistent user knowledge (long-term memory)
- ✅ Personalization across sessions

### **Next Steps:**

**Section 4** will add **tools** and **agentic workflows** using **LangGraph**, completing your journey from context engineering fundamentals to production-ready AI agents.

---

## 🎉 Congratulations!

You've successfully built a **memory-enhanced RAG system** that:
- Remembers conversations (working memory)
- Accumulates knowledge (long-term memory)
- Resolves references naturally
- Personalizes responses
- Integrates all four context types

**You're now ready for Section 4: Tools & Agentic Workflows!** 🚀

## 🧪 Hands-On: Working Memory in Action

Let's simulate a multi-turn conversation with working memory.


In [ ]:
# Working Memory Demo
async def working_memory_demo():
    """Demonstrate working memory for conversation continuity"""

    if not MEMORY_SERVER_AVAILABLE:
        print("⚠️  Memory Server not available. Skipping demo.")
        return

    student_id = "sarah_chen"
    session_id = f"session_{student_id}_demo"

    print("=" * 80)
    print("🧪 WORKING MEMORY DEMO: Multi-Turn Conversation")
    print("=" * 80)

    # Turn 1: First query
    print("\n📍 TURN 1: User asks about a course")
    print("-" * 80)

    user_query_1 = "Tell me about CS401"

    # Load working memory (empty for first turn)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   Messages in working memory: {len(working_memory.messages)}")
    print(f"   User: {user_query_1}")

    # Search for course
    courses = course_manager.search(user_query_1, limit=1)

    # Generate response (simplified - no full RAG for demo)
    if courses:
        course = courses[0]
        response_1 = f"{course.course_code}: {course.title}. {course.description[:100]}..."
    else:
        response_1 = "I couldn't find that course."

    print(f"   Agent: {response_1}")

    # Save to working memory
    working_memory.messages.extend([
        MemoryMessage(role="user", content=user_query_1),
        MemoryMessage(role="assistant", content=response_1)
    ])

    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   ✅ Saved to working memory")

    # Turn 2: Follow-up with pronoun reference
    print("\n📍 TURN 2: User uses pronoun reference ('its')")
    print("-" * 80)

    user_query_2 = "What are its prerequisites?"

    # Load working memory (now has 1 exchange)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   Messages in working memory: {len(working_memory.messages)}")
    print(f"   User: {user_query_2}")

    # Build context with conversation history
    messages = [
        SystemMessage(content="You are a helpful course advisor. Use conversation history to resolve references like 'it', 'that course', etc.")
    ]

    # Add conversation history from working memory
    for msg in working_memory.messages:
        if msg.role == "user":
            messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            messages.append(AIMessage(content=msg.content))

    # Add current query
    messages.append(HumanMessage(content=user_query_2))

    # Generate response (LLM can now resolve "its" using conversation history)
    response_2 = llm.invoke(messages).content

    print(f"   Agent: {response_2}")

    # Save to working memory
    working_memory.messages.extend([
        MemoryMessage(role="user", content=user_query_2),
        MemoryMessage(role="assistant", content=response_2)
    ])

    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   ✅ Saved to working memory")

    # Turn 3: Another follow-up
    print("\n📍 TURN 3: User asks another follow-up")
    print("-" * 80)

    user_query_3 = "Can I take it next semester?"

    # Load working memory (now has 2 exchanges)
    _, working_memory = await memory_client.get_or_create_working_memory(
        session_id=session_id,
        user_id=student_id,
        model_name="gpt-4o"
    )

    print(f"   Messages in working memory: {len(working_memory.messages)}")
    print(f"   User: {user_query_3}")

    # Build context with full conversation history
    messages = [
        SystemMessage(content="You are a helpful course advisor. Use conversation history to resolve references.")
    ]

    for msg in working_memory.messages:
        if msg.role == "user":
            messages.append(HumanMessage(content=msg.content))
        elif msg.role == "assistant":
            messages.append(AIMessage(content=msg.content))

    messages.append(HumanMessage(content=user_query_3))

    response_3 = llm.invoke(messages).content

    print(f"   Agent: {response_3}")

    print("\n" + "=" * 80)
    print("✅ DEMO COMPLETE: Working memory enabled natural conversation flow!")
    print("=" * 80)

# Run the demo
await working_memory_demo()


### 🎯 What Just Happened?

**Turn 1:** User asks about CS401
- Working memory: **empty**
- Agent responds with course info
- Saves: User query + Agent response

**Turn 2:** User asks "What are **its** prerequisites?"
- Working memory: **1 exchange** (Turn 1)
- LLM resolves "its" → CS401 (from conversation history)
- Agent answers correctly
- Saves: Updated conversation

**Turn 3:** User asks "Can I take **it** next semester?"
- Working memory: **2 exchanges** (Turns 1-2)
- LLM resolves "it" → CS401 (from conversation history)
- Agent answers correctly

**💡 Key Insight:** Working memory enables **reference resolution** and **conversation continuity**.

---


